In [4]:
import os
import openai
import json
import re
import difflib

In [2]:
# The documentation suggests that the organization string is probably safe to be in the source code.
# The key is secret and of course is not safe to include in code.
# Both are NOT in the code for now.
openai.organization = os.getenv("openapi_org")
openai.api_key = os.getenv("openapi_key")

In [3]:
# Probably the simplest call is to list the available models.
openai_models = openai.Model.list()
openai_models_json = json.loads(str(openai_models))
openai_models_json['data'][0]
available_models = []
for rec in openai_models_json['data']:
    available_models.append(rec['id'])
print("Here are all of the models:\n", available_models)

code_regex = re.compile("code")
code_models = list(filter(code_regex.search, available_models))
print("\nThese models may be useful for tasks with code:\n", code_models)


Here are all of the models:
 ['babbage', 'ada', 'davinci', 'text-embedding-ada-002', 'babbage-code-search-code', 'text-similarity-babbage-001', 'text-davinci-001', 'curie-instruct-beta', 'babbage-code-search-text', 'babbage-similarity', 'curie-search-query', 'code-search-babbage-text-001', 'code-cushman-001', 'code-search-babbage-code-001', 'text-ada-001', 'code-davinci-002', 'text-similarity-ada-001', 'text-davinci-insert-002', 'ada-code-search-code', 'text-davinci-002', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'text-curie-001', 'text-davinci-edit-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'code-davinci-edit-001', 'davinci-instruct-beta', 'text-babbage-001', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-similarity', 'davinci-search-query', 'text-davinci-insert-001', 'babbage-search-document', 'ada-search-document', 'curie', 'text-search-babb

In [11]:
# The model ids could easily be mistyped. Here is a helper function to find the closest to a user input.
def closest_model_id(models, user_input):
    best_guess = difflib.get_close_matches(user_input, models, cutoff=0.8, n=1)
    return(best_guess)

closest_model_id(available_models, "text-davinci-03")

['text-davinci-003']

In [18]:
# Let's see if the ai knows the history of the bioinformatics field.
bioinformatics_history_prompt = "Write a paragraph explaining the history of the bioinformatics field."
bioinformatics_history_model = closest_model_id(available_models, "text-davinci-003")
# TODO: determine how to pass these variables into the openai.Completion.create call
print("Prompt:\n", bioinformatics_history_prompt, "\nModel:\n", bioinformatics_history_model)
bioinformatics_history_response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Write a paragraph explaining the history of the bioinformatics field.",
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
)



print("\nResponse:\n", bioinformatics_history_response)

Prompt:
 Write a paragraph explaining the history of the bioinformatics field. 
Model:
 ['text-davinci-003']

Response:
 {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nBioinformatics is a relatively young field that has come about as the result of the convergence of several disciplines, including biology, computer science, mathematics, and engineering. It began in the 1970s when scientists first started to use computers to store and analyze biological data. Since then, the field has grown rapidly, as new technologies and techniques have been developed to store and analyze ever larger and more complex data sets. Today, bioinformatics is used in many fields of research, from basic research in genetics and molecular biology, to medical research, to agriculture and food production, to the development of new drugs and therapies. It is becoming increasingly important for research and development in many areas, as our underst

In [27]:
print(json.loads(str(bioinformatics_history_response))['choices'][0]['text'].lstrip('\n'))
# TODO: format properly so that the lines don't break in the middle of words

Bioinformatics is a relatively young field that has come about as the result of the convergence of several disciplines, including biology, computer science, mathematics, and engineering. It began in the 1970s when scientists first started to use computers to store and analyze biological data. Since then, the field has grown rapidly, as new technologies and techniques have been developed to store and analyze ever larger and more complex data sets. Today, bioinformatics is used in many fields of research, from basic research in genetics and molecular biology, to medical research, to agriculture and food production, to the development of new drugs and therapies. It is becoming increasingly important for research and development in many areas, as our understanding of the complexity of life grows.


In [30]:
# Let's do some real bioinformatics.
translate_to_protein_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite a python script that uses biopython to translate a DNA sequence to protein.\n\"\"\"",
  temperature=0,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(translate_to_protein_response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nfrom Bio.Seq import Seq\nfrom Bio.Alphabet import IUPAC\n\ndna = Seq(\"ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG\", IUPAC.unambiguous_dna)\n\nprint(dna.translate())"
    }
  ],
  "created": 1674253048,
  "id": "cmpl-6atzM5mgBQPOxL0AP8jQqO9D53VNA",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 71,
    "prompt_tokens": 21,
    "total_tokens": 92
  }
}


In [31]:
print(json.loads(str(translate_to_protein_response))['choices'][0]['text'].lstrip('\n'))

from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)

print(dna.translate())


In [37]:
# The code doesn't appear to be dangerous, so let's try executing it.
exec(json.loads(str(translate_to_protein_response))['choices'][0]['text'].lstrip('\n'))

ImportError: Bio.Alphabet has been removed from Biopython. In many cases, the alphabet can simply be ignored and removed from scripts. In a few cases, you may need to specify the ``molecule_type`` as an annotation on a SeqRecord for your script to work correctly. Please see https://biopython.org/wiki/Alphabet for more information.

In [38]:
# Looks like this code recommendation is obsolete. Maybe we can copy + paste and remove the alphabet bit.
from Bio.Seq import Seq

dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")

print(dna.translate())

MAIVMGR*KGAR*


Now that is interesting - with a very minor simplification, the code runs properly. One interesting question would be why the AI chose to include a stop codon in the middle of the random sequence it generated.

In [45]:
convert_gff_to_bed_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite a python script that converts a GFF3 file to bed format.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(convert_gff_to_bed_response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nimport sys\n\ndef gff2bed(gff_file):\n    \"\"\"\n    Convert a GFF3 file to bed format.\n    \"\"\"\n    with open(gff_file, 'r') as gff:\n        for line in gff:\n            if line.startswith('#'):\n                continue\n            else:\n                line = line.strip().split('\\t')\n                chrom = line[0]\n                start = line[3]\n                end = line[4]\n                name = line[8].split(';')[0].split('=')[1]\n                strand = line[6]\n                print(chrom, start, end, name, '0', strand, sep='\\t')\n\nif __name__ == '__main__':\n    gff2bed(sys.argv[1])"
    }
  ],
  "created": 1675099154,
  "id": "cmpl-6eS6EUFrxrw3hmkIKtzHBcIv5MEUC",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 194,
    "prompt_tokens": 19,
    "total_tokens": 213
  }
}


In [46]:
print(json.loads(str(convert_gff_to_bed_response))['choices'][0]['text'].lstrip('\n'))

import sys

def gff2bed(gff_file):
    """
    Convert a GFF3 file to bed format.
    """
    with open(gff_file, 'r') as gff:
        for line in gff:
            if line.startswith('#'):
                continue
            else:
                line = line.strip().split('\t')
                chrom = line[0]
                start = line[3]
                end = line[4]
                name = line[8].split(';')[0].split('=')[1]
                strand = line[6]
                print(chrom, start, end, name, '0', strand, sep='\t')

if __name__ == '__main__':
    gff2bed(sys.argv[1])


This code has some interesting features.
1. It is a first-principles example that splits each GFF line on the tabs rather than reading it into a complex data structure with a GFF parser.
2. It does not understand that the start coordinate is 1-based in GFF and 0-based in BED, although a slightly different prompt I have tried does understand that.
3. It uses the first attribute of GFF column 9 as the BED "name" rather than explicitly finding something called "name" or "ID"; this could be misleading based on the input GFF.
4. It does not try to extract the "score" from the GFF, but rather just uses 0 for the BED file.
5. It chooses to write a 6 column BED file.
6. It will not handle blank lines in the GFF.

**The biggest problem is that each BED line is going to have an off-by-one error, and this example code is quite dangerous in that way.**

In [56]:
def first_openai_response(full_openai_response):
    r = json.loads(str(full_openai_response))['choices'][0]['text'].lstrip('\n')
    return r

In [69]:
convert_gff_to_bed_biopython_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite a python script that uses biopython to parse a GFF3 file and uses biopython to format each record in BED format with 0-based start coordinates.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [70]:
print(first_openai_response(convert_gff_to_bed_biopython_response))

import sys
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

def main(gff_file):
    for record in SeqIO.parse(gff_file, "gff"):
        for feature in record.features:
            if feature.type == "gene":
                print(record.id, feature.location.start, feature.location.end, feature.qualifiers["Name"][0], ".", feature.strand)

if __name__ == "__main__":
    main(sys.argv[1])


In [51]:
convert_gff_to_bed_shell_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite a bash script that converts a GFF3 file to bed format.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(convert_gff_to_bed_shell_response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nimport sys\n\ndef gff2bed(gff_file):\n    \"\"\"\n    Convert a GFF3 file to bed format.\n    \"\"\"\n    with open(gff_file) as f:\n        for line in f:\n            if line.startswith(\"#\"):\n                continue\n            else:\n                line = line.strip().split(\"\\t\")\n                chrom = line[0]\n                start = line[3]\n                end = line[4]\n                name = line[8].split(\";\")[0].split(\"=\")[1]\n                print(\"\\t\".join([chrom, start, end, name]))\n\nif __name__ == \"__main__\":\n    gff2bed(sys.argv[1])"
    }
  ],
  "created": 1675099600,
  "id": "cmpl-6eSDQkhq9wPpUFExrNZYteor1ynef",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 176,
    "prompt_tokens": 19,
    "total_tokens": 195
  }
}


In [52]:
print(first_openai_response(convert_gff_to_bed_shell_response))

import sys

def gff2bed(gff_file):
    """
    Convert a GFF3 file to bed format.
    """
    with open(gff_file) as f:
        for line in f:
            if line.startswith("#"):
                continue
            else:
                line = line.strip().split("\t")
                chrom = line[0]
                start = line[3]
                end = line[4]
                name = line[8].split(";")[0].split("=")[1]
                print("\t".join([chrom, start, end, name]))

if __name__ == "__main__":
    gff2bed(sys.argv[1])


Well, that is not ideal - it just wrote a Python script again instead of bash. I wonder if we can get it to use awk?

In [53]:
convert_gff_to_bed_shell_2_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite a bash script using awk that converts a GFF3 file to bed format.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)


In [55]:
print(first_openai_response(convert_gff_to_bed_shell_2_response))

import sys
import argparse

def main(args):
    with open(args.input, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            line = line.strip().split('\t')
            if line[2] == 'gene':
                print('\t'.join([line[0], line[3], line[4], line[8].split(';')[0].split('=')[1]]))

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument('input', help="Input GFF3 file")
    args = parser.parse_args()
    main(args)


Once again, more Python. The documentation does indicate that the code-davinci-002 model is most proficient with Python.

In [71]:
bioinformatics_coordinates_response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Explain the use of 0-based and 1-based start coordinates in the GFF3 and BED file formats",
  temperature=0.7,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
)

In [72]:
print(bioinformatics_coordinates_response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThe GFF3 and BED file formats are used to store genomic feature annotations. Both formats use 0-based start coordinates, meaning the first base of a feature is numbered 0 (e.g. the first base of a gene is 0). This is in contrast to 1-based start coordinates, where the first base of a feature is numbered 1. 0-based start coordinates are typically used in bioinformatic tools and analysis programs, as they are more intuitive for working with arrays, strings, and other numerical data structures."
    }
  ],
  "created": 1675100566,
  "id": "cmpl-6eST0hyLUhX5c8gQ3yuMUY7HiFWYn",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 110,
    "prompt_tokens": 24,
    "total_tokens": 134
  }
}


This is incorrect. GFF3 uses 1-based coordinates.

In [73]:
convert_gff_to_bed_3_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWrite 3 lines of GFF3 and convert them to BED.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [75]:
print(first_openai_response(convert_gff_to_bed_3_response))

import sys
import os
import unittest

from pybedtools import BedTool
from pybedtools.contrib.gff3 import GFF3_HEADER

from . import GFF3_BED_HEADER
from . import GFF3_BED_FOOTER
from . import GFF3_BED_LINE1
from . import GFF3_BED_LINE2
from . import GFF3_BED_LINE3
from . import GFF3_BED_LINE4
from . import GFF3_BED_LINE5
from . import GFF3_BED_LINE6
from . import GFF3_BED_LINE7
from . import GFF3_BED_LINE8
from . import GFF3_BED_LINE9
from . import GFF3_BED_LINE10
from . import GFF3_BED_LINE11
from . import GFF3_BED_LINE12
from . import GFF3_BED_LINE13
from . import GFF3_BED_LINE14
from . import GFF3_BED_LINE15
from . import GFF3_BED_LINE16
from . import GFF3_BED_LINE17
from . import GFF3_BED_LINE18
from . import GFF3_BED_LINE19
from . import GFF3_BED_LINE20
from . import GFF3_BED_LINE21
from . import GFF3_BED_LINE22
from . import GFF3_BED_LINE23
from . import GFF3_BED_LINE24
from . import GFF3_BED_LINE25
from . import GFF3_BED_LINE26
from . import GFF3_BED_LINE27
from . import GFF3_BE

That appears to have throughly confused the AI. Perhaps a code model can't fabricate GFF3?

In [78]:
align_protein_sequence_1_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nCreate a protein sequence consisting of length 50 and align to the human genome with python.\n\"\"\"",
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [82]:
print(first_openai_response(align_protein_sequence_1_response))

import random
import re
import sys

# Create a random protein sequence of length 50
protein = ""
for i in range(50):
    protein += random.choice("ACDEFGHIKLMNPQRSTVWY")

# Read in the human genome
genome = ""
with open("human_genome.txt", "r") as f:
    for line in f:
        genome += line.strip()

# Find all matches of the protein sequence in the genome
matches = re.finditer(protein, genome)

# Print the locations of the matches
for match in matches:
    print(match.start())


In [90]:
align_protein_sequence_2_response = openai.Completion.create(
  model="code-davinci-002",
  prompt="\"\"\"\nWith Python, create a random protein sequence of length 50 called 'randprot', then use Biopython to read in a FASTA file containing the human genome, then use BLAST to align 'randprot' to the human genome.\n\"\"\"",
  temperature=0,
  max_tokens=912,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [91]:
print(first_openai_response(align_protein_sequence_2_response))

from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
import random

# Create a random protein sequence of length 50
randprot = Seq("".join(random.choice("ACDEFGHIKLMNPQRSTVWY") for i in range(50)), IUPAC.protein)

# Read in the human genome
human_genome = SeqIO.read("human_genome.fasta", "fasta")

# Use BLAST to align 'randprot' to the human genome
result_handle = NCBIWWW.qblast("blastp", "nr", randprot)

# Save the output to a file
save_file = open("my_blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()
